In [15]:
import pandas as pd
import numpy as np

In [16]:
sp500 = pd.read_csv("SP500/sp500_1.csv")
# Rename columns
sp500.columns = ['Date', 'sp500', 'sp500+1','ret_sp500', 'ret_sp500+1']
sp500 = sp500.dropna()
sLength = len(sp500['Date'])
# add the bullMarket column with random ints (to be replaced in next cell)
sp500['bullMarket'] = pd.Series(np.random.randn(sLength), index=sp500.index)

# Create bullMarkert for ret_sp500+1
bull = []
for data in sp500['ret_sp500+1']:
    if data > 0:
        bull.append(1)
    else:
        bull.append(0)
sp500['bullMarket'] = bull
print(sp500)

sp500.shape
# print(sp500)

          Date    sp500  sp500+1  ret_sp500  ret_sp500+1  bullMarket
1       1/1/64    75.02    75.43   0.000000     0.546521           1
2       1/2/64    75.43    75.50   0.546521     0.092801           1
3       1/3/64    75.50    75.67   0.092801     0.225166           1
4       1/6/64    75.67    75.69   0.225166     0.026431           1
5       1/7/64    75.69    76.00   0.026431     0.409565           1
6       1/8/64    76.00    76.28   0.409565     0.368421           1
7       1/9/64    76.28    76.24   0.368421    -0.052438           0
8      1/10/64    76.24    76.22  -0.052438    -0.026233           0
9      1/13/64    76.22    76.36  -0.026233     0.183679           1
10     1/14/64    76.36    76.64   0.183679     0.366684           1
11     1/15/64    76.64    76.55   0.366684    -0.117432           0
12     1/16/64    76.55    76.56  -0.117432     0.013063           1
13     1/17/64    76.56    76.41   0.013063    -0.195925           0
14     1/20/64    76.41    76.62  

(14275, 6)

In [30]:
tdata = pd.read_csv("Total_Data_no_dots.csv",low_memory=False)

result = pd.merge(sp500, tdata, on='Date', how='left')

result  = result.dropna()

# assume we want to predict the raw index of sp500 at T+1, given all the info at T

y_reg = result['ret_sp500+1']

y_cal = result['bullMarket']
 
# list(result)

result = result.drop(columns=['VOX Close', 'DGS1', 'DGS10', 'T10Y2Y', 'DBAA', 'USD1MTD156N', 'DCOILBRENTEU', 'DTWEXB', 'DEXUSEU', 'T10YIE', 'T5YIFR', 'BAMLH0A0HYM2EY'])

x = result.drop(columns=['Date','sp500_x','sp500+1','ret_sp500+1','Unnamed: 0'])

# print(result.dtypes)

In [31]:
# Linear Regression Model

from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.cross_validation import KFold

# this is just an example
# x = result[['sp500_x','VIX Close']]

kf = KFold(x.shape[0], n_folds=5, shuffle=True)

for train_index, test_index in kf:
    x_train, x_test = x.iloc[train_index], x.iloc[test_index]
    y_train, y_test = y_reg.iloc[train_index], y_reg.iloc[test_index]

    regr = linear_model.LinearRegression()
    regr.fit(x_train, y_train)

    y_pred = regr.predict(x_test)
    print("MSE: ", mean_squared_error(y_test, y_pred))

MSE:  0.8005489780596978
MSE:  0.9110931089470364
MSE:  0.7155102661319641
MSE:  0.7787915934620637
MSE:  0.7945047010746361


In [32]:
# clssification tasks

# tree 

from sklearn import tree
clf = tree.DecisionTreeClassifier()
clf = clf.fit(x, y_cal)
predicted = clf.predict(x)
# print (predicted)

# naive Bayes 

from sklearn.naive_bayes import GaussianNB

gnb = GaussianNB()

gnb.fit(x, y_cal).predict(x)

predicted= gnb.predict(x)
print (predicted)

from sklearn.metrics import confusion_matrix

print(confusion_matrix(predicted,y_cal))  


[1 1 1 ... 1 1 1]
[[ 394  364]
 [3830 4160]]
